In [1]:
import pandas as pd
import sys
import requests
from xlsxwriter.utility import xl_rowcol_to_cell
sys.path.append(r'/Users/stepwate/Python Codes/Reusable Code')
import google_sheets_py as gspy
import trello_generic as tg
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)


from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
import itertools

from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import pacf


### Import FTS data into a data frame

In [22]:

# Google Sheets Credentials
credentials_path='/Users/stepwate/Python Codes/Reusable Code/Credentials to Use/'
gspy.authenticate_sheets(credentials_path, credentials_file_name='credentials.json',scope='readwrite')
FTS=gspy.read_google_sheets_as_rows('1e--ETf-JLnQe-VLfwZP0RSiNRrzP9QbdAUHHFc5Bm3c','Data sheet 4 hourly',credentials_path)




In [23]:
# As row headers are first entry in list of lists, transform to a dictionary format which is better read by pandas
row_dict={}
for n,y in enumerate(FTS[0]):
    row_dict[y]=[x[n] for i,x in enumerate(FTS) if i>0]

FTS_temp =pd.DataFrame(row_dict)  

FTS_temp = FTS_temp[['starting_hour','N_custs']]

# Map reporting date as date time in correct format
FTS_temp['reporting date']=pd.to_datetime(FTS_temp['starting_hour'],format='%d/%m/%Y %H:%M:%S')

# Set reporting date as index too to enable various pandas functions
FTS_temp.index=FTS_temp['reporting date']

# Map variable types
FTS_temp=FTS_temp.astype({'N_custs':'float64'})


FTS_df = pd.DataFrame(FTS_temp['N_custs']).rename(columns={'N_custs':'N'})


In [24]:
FTS_df

# 1) Create different datasets
Once the data is in the format of a single column with the time/date values in the index, you can split it into three components:
1) The data used to train the model
2) The data used to evaluate the model
3) The "actuals" for the period you forecast.

Note that a set of data is better used for evaluation (2) because the nature of time series is that they use past observations. Therefore after a few forecast periods foreward, they tend to converge on a repeating pattern as there is no "real" data to show variance. As such a model can fit extremely well but still veer off when used for forecasting. By using it to model e.g. a week of known values, we can get a sense of how much it diverges.

3) is only relevant where you are trying to measure uplift by comparing observed vs expected. If doing a genuine forecast into the future, there will be no "actuals"

In [130]:
def parse_datasets(complete_df, training_start, forecast_start, forecast_end, seasonality_period\
                   ,eval_duration=(1,'s'),training_end=None):
    
    # Aim is to create three datasets:
    # Training (the period to build the model off of)
    # Eval actuals (a period just before the model is build on which to test the model fit)
    # Forecast actuals (a period that you're going to predict. Actuals will only exist if you are using it for incremental analysis)
    
    # If NO training_end specified, the training dataset and the evaluation dataset will be mutually exclusive, 
    # i.e. training will end where eval begins. If it is explicitly specified, there will be some overlap
    
    # Remove the section to be forecasted
    partial_df=complete_df[training_start:forecast_start][:-1]
    try:
        forecast_actuals=complete_df[forecast_start:forecast_end]
    except:
        forecast_actuals=None
    
    #
    if eval_duration[1]=='s':
        if training_end==None:
            training_df=partial_df[:-((eval_duration[0]*seasonality_period))]
        else:
            training_df=partial_df[:training_end]
            
        eval_actuals=partial_df[-((eval_duration[0]*seasonality_period)):]
    
    else:
        if training_end==None:
            training_df=partial_df[:-((eval_duration[0]))]
        else:
            training_df=partial_df[:training_end]
        eval_actuals=partial_df[-((eval_duration[0])):]

    return (training_df,eval_actuals,forecast_actuals)

def model_fit_results(df,observed_col,forecast_col=None,resid_col=None,showplot=True):
    if resid_col and forecast_col:
        df['modelled'] = df[forecast_col]
        df['Error']=df[resid_col]
    elif resid_col:
        df['Error']=df[resid_col]
        df['modelled'] = df[observed_col]-df['Error']        
    elif forecast_col:
        df['modelled'] = df[forecast_col]
        df['Error']=df[observed_col]-df['modelled']
    else:
        print("Error: you need either the forecasted values or the residuals to show model fit results")
        return
    
df['Squared error']=df['Error']**2
df['Absolute error']=abs(df['Error'])
df['Percentage error']=df['Error']/ df[observed_col]
df['Absolute Percentage error']=df['Absolute error']/df[observed_col]

    results=df[['Error','Squared error','Absolute error','Percentage error','Absolute Percentage error']].mean()
    if showplot==True:
        df[[observed_col,'modelled']].plot()
        pyplot.show() 
    return results.to_dict()
    


def Fit_various_models(full_df,training_start,forecast_start,forecast_end,seasonality_lag,eval_periods,frequency='d'):
    
    var_to_model=full_df.columns[0]
    
    #Call the previously defined "parse datasets" function to return the full_df split into components for fitting
    # model and forecasting
    training, eval_actuals, forecast_actuals=parse_datasets(full_df,training_start,forecast_start\
                                                        ,forecast_end ,seasonality_lag,eval_duration=(eval_periods,'d'))

    
    # Initialise a dictionary holding all model results
    all_model_fits=[]
        
    ####### Part 1: Fit a combination of Holt Winters Models, looping through each version- additive, multiplicative 
    ####### and those same options for seasonal, or no seasonality at all
    
    hw_combos = list(itertools.product(['add','mul'],['add','mul','None'])) #Create all 6 combos of additive, multiplicative and True/False
    
    for n,model_var in enumerate(hw_combos):
        print("Trying Holt Winter's Model {}, model number {}".format(model_var,n))
        try:
            
            # Creates the time series model using the ExponentialSmooothing function
            if model_var[1]=='None':
                model=ExponentialSmoothing(training, trend=model_var[0],freq=frequency)
 
            else:
                model=ExponentialSmoothing(training, trend=model_var[0],seasonal=model_var[1]\
                                           ,seasonal_periods=seasonality_lag,freq=frequency)
                
            print("Model fitted")
            # Stores the model fit attribute
            model_fit=model.fit()
            model_fit.summary() # Provides summary statistics
            
            ### Evaluation of accuracy in predicting the data in the "evaluation" period
            
            #Produce a 'forecast' for the evaluation time period
            eval_actuals['estimate']=model_fit.predict(start=eval_actuals.index.min(), end=eval_actuals.index.max())
            my_eval=model_fit_results(eval_actuals,var_to_model,forecast_col='estimate',showplot=False)
            all_model_fits.append({'model':"Holt Winter's {}".format(model_var),\
                                   'AIC':model_fit.aic,\
                                  'avg error':my_eval['Error'],\
                                   'avg Squared error':my_eval['Squared error'],\
                                   'avg Absolute error':my_eval['Absolute error'],\
                                   'avg Percentage error':my_eval['Percentage error'],\
                                   'avg Absolute Percentage error':my_eval['Absolute Percentage error']\
                                  })
            
            
            
            
            
            
            #eval_estimated=pd.DataFrame(model_fit.predict(start=eval_actuals.index.min(), end=eval_actuals.index.max()))
            # Join on the actuals so there is a dataframe with real and estimated values
            #full_eval_df=eval_actuals.join(eval_estimated[0]).rename(columns={var_to_model:'Actual',0:'Estimated'})
            # Call the function above to return the fit of it
            

        except: 
            print("Model {} Failed".format(model_var))
    
    
    ####### Part 2: Fit a range of ARIMA models
    
    # Define the p, d and q parameters to take any value between 0 and 2
    p= P = range(0,3)
    d = q = D=Q= range(0, 2)
    s=[seasonality_lag,0]
    # Generate all different combinations of p, q and q triplets
    pdq = list(itertools.product(p, d, q))
    PDQ = list(itertools.product(P,D,Q,s))
    arima_combos=list(itertools.product(pdq,PDQ))
    
    

   
    # Loop through models 
    for n,model_var in enumerate(arima_combos):
        print("Trying ARIMA {}, model number {}".format(model_var,n))
        if model_var[0]==(0,0,0):
            pass
        else:
            try:  
                model=SARIMAX(training, order=model_var[0], seasonal_order=model_var[1],freq=frequency, simple_differencing=True)
                # Stores the model fit attribute
                model_fit=model.fit()
                model_fit.summary() # Provides summary statistics

                ### Evaluation of accuracy in predicting the data in the "evaluation" period

                #Produce a 'forecast' for the evaluation time period
                eval_actuals['estimate']=model_fit.predict(start=eval_actuals.index.min(), end=eval_actuals.index.max())
                my_eval=model_fit_results(eval_actuals,var_to_model,forecast_col='estimate',showplot=False)

                all_model_fits.append({'model':"ARIMA {}".format(model_var),\
                                       'AIC':model_fit.aic,\
                                      'avg error':my_eval['Error'],\
                                       'avg Squared error':my_eval['Squared error'],\
                                       'avg Absolute error':my_eval['Absolute error'],\
                                       'avg Percentage error':my_eval['Percentage error'],\
                                       'avg Absolute Percentage error':my_eval['Absolute Percentage error']\
                                      })


            except:
                print("Model {} Failed".format(model_var))
            
            
    best_AIC=[i for i in all_model_fits if i['AIC']==min([i['AIC'] for i in all_model_fits])][0]
    print(' The best AIC was on model : {} at a value of {}. The MAPE was {}'\
              .format(best_AIC['model'],best_AIC['AIC'],best_AIC['avg Absolute Percentage error']))
        
    best_MAPE=[i for i in all_model_fits if i['avg Absolute Percentage error']==min([i['avg Absolute Percentage error'] for i in all_model_fits])][0]
    print(' The best MAPE was on model : {} at a value of {}. The MAPE was {}'\
              .format(best_MAPE['model'],best_MAPE['AIC'],best_MAPE['avg Absolute Percentage error']))
    
    pd.DataFrame(all_model_fits).plot.scatter(x='avg Absolute Percentage error',y='AIC')
    pyplot.show()
    return all_model_fits
  
       

In [131]:
# Correlation with itself (yesterday)
print("Correlation at lag 1 is :",FTS_df['N'].autocorr())

# Correlation with itself (7 days ago)
print("Correlation at lag 24 is :",FTS_df['N'].autocorr(24))

# Correlation with exactly same record (useless sanity check)
print("Correlation at lag 168 is :",FTS_df['N'].autocorr(168))

# Plot autoregression to see how it correlates with each lag
autocorrelation_plot(FTS_df['N'])
pyplot.show()

# Get Partial ACF

PACF_values=pd.DataFrame(pacf(FTS_df['N'],nlags=200)) #PACF numpy array converted to df
PACF_values.plot() #Plot
pyplot.show()

# In this example it appears a PACF can be >1. We have some erratic values going back beyond 25 instances which mask the pattern
sub_df=PACF_values[0:1000]
sub_df.plot(style='.-',xlim=(0,26))
pyplot.show()

PACF_values

In [193]:
#training_start_date='2019-11-11'
#forecast_start_date='2020-02-29 19:00:00'
#forecast_end_date='2020-03-03 23:00:00'

my_training, my_eval_actuals, my_forecast_actuals=parse_datasets(FTS_df,'2019-11-11','2020-02-29 16:00:00',\
                                                                 '2020-03-03 23:00:00',42,eval_duration=(42,'d'))


In [38]:
results6=Fit_various_models(FTS_df,'2019-11-11','2020-02-29 16:00:00','2020-03-03 23:00:00',6,42,frequency='4h')

In [40]:
results6_df=pd.DataFrame(results6)
results6_df[(results6_df['AIC']<6200) &(results6_df['avg Absolute Percentage error']<.2)]\
.sort_values(by='avg Absolute Percentage error')

In [71]:
#I tried 21 because of the PACF but results were garbage. 
results42=Fit_various_models(FTS_df,'2019-11-11','2020-02-29 16:00:00','2020-03-03 23:00:00',42,168,frequency='4h')

In [75]:
results42_df=pd.DataFrame(results42)
results42_df[(results42_df['avg Absolute Percentage error']<.5)]\
.sort_values(by='avg Absolute Percentage error')

In [157]:
results6[1]

In [210]:

# Select the best models and store them
best_model6=ExponentialSmoothing(my_training, trend='add',seasonal='mul',seasonal_periods=6,freq='4h')
best_model6v2=ExponentialSmoothing(my_training, trend='add',seasonal='mul',seasonal_periods=6,freq='4h')

best_model42=SARIMAX(my_training, order=(0,0,1), seasonal_order=(1,0,1,42),freq='4h', simple_differencing=True)


# Add evaluation period data back into the model
training_full_df=pd.DataFrame(my_training['N'].append(my_eval_actuals['N']))


best_model6_refitted=ExponentialSmoothing(my_training, trend='add',seasonal='mul',seasonal_periods=6,freq='4h').fit(\
smoothing_level=best_model6.fit().params['smoothing_level'],\
smoothing_slope=best_model6.fit().params['smoothing_slope'],\
smoothing_seasonal=best_model6.fit().params['smoothing_seasonal'],\
damping_slope=best_model6.fit().params['damping_slope'],\
use_boxcox=best_model6.fit().params['use_boxcox'],\
#start_params= best_model6.fit().params['initial_seasons'],\
remove_bias=best_model6.fit().params['remove_bias'],\
initial_level=best_model6.fit().params['initial_level'],\
#initial_seasons=best_model6.fit().params['initial_seasons'],\
initial_slope=best_model6.fit().params['initial_slope'])


best_model6_refitted_full=ExponentialSmoothing(training_full_df, trend='add',seasonal='mul',seasonal_periods=6,freq='4h').fit(\
smoothing_level=best_model6.fit().params['smoothing_level'],\
smoothing_slope=best_model6.fit().params['smoothing_slope'],\
smoothing_seasonal=best_model6.fit().params['smoothing_seasonal'],\
damping_slope=best_model6.fit().params['damping_slope'],\
use_boxcox=best_model6.fit().params['use_boxcox'],
remove_bias=best_model6.fit().params['remove_bias'],\
initial_level=best_model6.fit().params['initial_level'],\
                                                                                                                         
initial_slope=best_model6.fit().params['initial_slope'])


best_model42_refitted=SARIMAX(my_training, order=(0,0,1), seasonal_order=(1,0,1,42),freq='4h', simple_differencing=True)
best_model42_refitted_full=SARIMAX(training_full_df, order=(0,0,1), seasonal_order=(1,0,1,42),freq='4h', simple_differencing=True)



In [211]:
print(best_model42.fit().params)
print(best_model42_refitted.fit().params)
print(best_model42_refitted_full.fit().params)

In [177]:
print(best_model6.fit().params)
print(best_model6v2.fit().params)
print(best_model6_refitted.params)


In [198]:
best_model6_refitted.summary()


In [200]:
best_model6.fit().summary()

In [215]:
# Want all values to be the same because the training data set is always the same (haven't used the new one yet)
Forecast_df=my_forecast_actuals.copy()
Forecast_df['best6']=best_model6.fit().predict(start='2020-02-29 16:00:00',end='2020-03-03 20:00:00')
Forecast_df['best6 refitted to training']=best_model6_refitted.predict(start='2020-02-29 16:00:00',end='2020-03-03 20:00:00')
Forecast_df['best6 dupe']=best_model6v2.fit().predict(start='2020-02-29 16:00:00',end='2020-03-03 20:00:00')
Forecast_df['best6 refitted to full']=best_model6_refitted_full.predict(start='2020-02-29 16:00:00',end='2020-03-03 20:00:00')
Forecast_df['best42 ARIMA']=best_model42_refitted_full.fit().predict(start='2020-02-29 16:00:00',end='2020-03-03 20:00:00')
Forecast_df

In [216]:

pd.set_option('display.max_rows', 700)
training2=my_training.copy()
training2['best6 fitted']=best_model6.fit().fittedvalues
training2['best6 resid']=best_model6.fit().resid
training2['best6 dupe fitted']=best_model6v2.fit().fittedvalues
training2['best6 dupe resid']=best_model6v2.fit().resid
training2['best6 refitted fitted']=best_model6_refitted.fittedvalues
training2['best6 refitted resid']=best_model6_refitted.resid
training2['best42 refitted fitted']=best_model42_refitted_full.fit().fittedvalues
training2['best42 refitted resid']=best_model42_refitted_full.fit().resid

training2

In [219]:
# Select best 42 model

final=training_full_df.copy()
final['expected']=best_model42_refitted_full.fit().fittedvalues


Forecast=my_forecast_actuals.copy()
Forecast['expected']=best_model42_refitted_full.fit().predict(start='2020-02-29 16:00:00',end='2020-03-03 20:00:00')


final['2020-02-01':].append(Forecast)

### Fit model to latest data

In [78]:
model=SARIMAX(my_training, order=(0,0,1), seasonal_order=(1,0,1,42),freq='4h', simple_differencing=True)
#model=ExponentialSmoothing(my_training, trend='add',seasonal='mul'\
#                                           ,seasonal_periods=42,freq='4h')
                
# Stores the model fit attribute
model_fit=model.fit()
print(model_fit.summary()) # Provides summary statistics

                ### Evaluation of accuracy in predicting the data in the "evaluation" period

                #Produce a 'forecast' for the evaluation time period
my_eval_actuals['estimate']=model_fit.predict(start=my_eval_actuals.index.min(), end=my_eval_actuals.index.max())
my_eval=model_fit_results(my_eval_actuals,'N',forecast_col='estimate',showplot=True)


In [68]:
to_do_forecast_df=pd.DataFrame(my_training['N'].append(my_eval_actuals['N']))
to_do_forecast_df

In [107]:
model_fit.aic

In [113]:
model_fit.params

In [125]:
newly_fitted=ExponentialSmoothing(to_do_forecast_df).fit(\
smoothing_level=model_fit.params['smoothing_level'],\
smoothing_slope=model_fit.params['smoothing_slope'],\
smoothing_seasonal=model_fit.params['smoothing_seasonal'],\
damping_slope=model_fit.params['damping_slope'],\
use_boxcox=model_fit.params['use_boxcox'],\
remove_bias=model_fit.params['remove_bias'],\
initial_level=model_fit.params['initial_level'],\
initial_slope=model_fit.params['initial_slope'])

newly_fitted.predict(start='2020-02-29 16:00:00',end='2020-03-03 20:00:00')

In [62]:
my_forecast_actuals['expected']=model_fit.predict(start='2020-02-29 16:00:00',end='2020-03-03 20:00:00')
my_forecast_actuals.plot()

In [63]:
my_forecast_actuals

In [64]:
### I have two models- one trained up to 22nd, other trained up to 29th.
### I want the model trained up to 22nd in terms of parameters etc., as that was the one evaluated
###, but modelling using data up to 29th

In [93]:
model_fit.alpha

In [88]:
help(ExponentialSmoothing.from_formula)
#(model_fit,my_forecast_actuals)


In [54]:
my_forecast_actuals['expected']=model_fit.predict(start='2020-03-01 20:00:00',end='2020-03-03 20:00:00')
my_forecast_actuals.plot()

## Describing Data
#### Autocorrelation 
is the correlation between a given point in time Tx and a previous point in time Tx-n . Autocovariance is related in the usual way covariance and correlation are.
The AutoCorrelation Function is the set of autocorrelations i.e. where n=1 to n=infinity

#### Decomposition 
is the process of breaking a time series into component parts, namely:
##### $Y_t$ (observed value at time T)
##### $S_t$ (seasonal component at time T)
##### $T_t$ (trend cycle component at time T)
##### $ε_t$ (irregular or error component at time T)


#### Additive vs Multiplicative Decomposition 
We have additive decomposition if:
$Y_t = S_t + T_t + ε_t$

and we have additive decomposition if:
$Y_t = S_t * T_t * ε_t$


#### Smoothing for moving averages
The first step in decomposing a time series is to estimate the trend cycle Tt. This is done using weighted moving averages. A traditional, simple, moving average can be described as a weighted-moving average whereby all the weights are the same. Weighted ones would typically weight the current observation and those nearest in time to it ($n+1$ or $n-1$) higher than those further from it (e.g. $n-5$). 
For analysis purposes it is ok to use the subsequent observations in a moving average, i.e. the window can look forwards as well as backwards to isolate the trend component.

#### Calculating seasonal component
You can subtract the Trend component, $T_t$ (i.e. the moving average part) from the overall observed $Y_t$ to be left with $ S_t + ε_t$ . The $S_t$ can then be thought of as the average of each seasonal period e.g. the average value seen in December (for multi-year trends) or the average "Monday" for weekly patterns. Importantly, you want the average of the Seasonal+Error ($ S_t + ε_t$ ), not the overall $Y_t$.
This average becomes the seasonal part
The remaining part, error, $ε_t$ should theoretically be randomly distributed. However in real-world stuff I'm not sure how that works as e.g. you would expect Marketing to provide "Error" to the model- but structured Error.

N.B. If building a multiplicative rather than an additive model, you would divide the Trend component to get a ratio, rather than subtract.



## Forecasting Data
The above helps describe what we've seen but doesn't provide a means of forecast, or even a means of determining uplift e.g. for Campaign Eval purposes (as that uplift will be baked into the Moving Average Trend).


The simplest forecast is a moving average, whereby $Y_{t+1}$ is an average of $Y_t$ back to $Y_{t-n}$. 

#### Stationarity
Moving averages smooth over trends and seasonal differences, so are a very crude forecast. You can also only forecast one step ahead. 
Effectively they are only useful if data is "Stationary"- which means that $Y_t$ is not dependent on $t$. A stationary data set does not contain a trend or seasonal component.


### Error evaluations
- ME: Mean Error (average error)
- MAE: Mean Absolute Error
- MSE: Mean Squared Error (square first then take the mean of it!)
- MPE: Mean Percentage error (as a % of the observed)
- MAPE: Mean Absolute Percentage Error

Also consider...:
- Max Error

Note: assuming errors are normally distributed, which in theory they should be, then $F_{t+1} +/- Z* \surd (MSE)$ gives approximate confidence intervals.

$Z= 1.645$ for 90%, $Z=1.960$ for 95% and $Z=2.576$  for 99%

In [ ]:
Fit_various_models(FTS_df,'2019-11-11','2020-02-29 19:00:00','2020-03-03 23:00:00',168,frequency='h')

In [ ]:

def Fit_Selected_HW(full_df,training_start,forecast_start,forecast_end,var_to_forecast,periods_seasonality):
    
    model = ExponentialSmoothing(training_df, trend='mul',seasonal='mul',seasonal_periods=periods_seasonality)
    model_fit=model.fit()
    
    training_df['residuals'] = pd.DataFrame(model_fit.resid)
    my_eval=model_fit_results(training_df,var_to_forecast,resid_col='residuals')
    print(my_eval)
    return training_df, model_fit

def Fit_Selected_ARIMAs(full_df,training_start,forecast_start,forecast_end,var_to_forecast,pdq,PDQS):
    
    training_df=full_df[training_start:forecast_start][[var_to_forecast]][:-1] # Create a df without the forecasted values
    model=SARIMAX(training_df, order=pdq, seasonal_order=PDQS)
    model_fit=model.fit()
    training_df['residuals'] = pd.DataFrame(model_fit.resid)
    my_eval=model_fit_results(training_df,var_to_forecast,resid_col='residuals')
    return training_df, model_fit


## Single Expontential Smoothing
Where F is the forecast value:

$F_{t+1} = αY_t + (1-α) F_t$

It can only be used to predict one step ahead. Effectively it combined a weighted blen of the current observation and the forecasted value of the current observation (which in turn, was dependent on the previous value, and so on.
A high α value means that the current actual is a powerful preditor, and the "history" less so. A low α basically upweights prior forecasts, which are products of previous observations.

The model must be initialised, as obviously there is no F1. There are lots of things you can do but a good place to start is $F_2=Y_1$ (i.e. forecast for period 2= observed for period 1).

You would iterate over lots of values of α such that you minimised the mean squared error between observed and forecast values.


## Holt's Linear Exponential Smoothing
An extension of exponential smoothing to account for a local, linear trend. The trend makes it possible to forecast m time period ahead. There are two smoothing constants, α and β.

## Holt- Winter's Method: Additive & Multiplicative
These extend the models further to also include Seasonality, where k= the number of periods in one cycle of seasons e.g. 7 days in a week or 12 months in a year.
To initialise, we need one complete cycle of data, $s$. E.g. for weekly seasonality we need at least 7 values, obviously. 
Further to this, to initialise trend, we need $s+k$ observations. This is because for trend there needs to be some sense of change. At absolute minimum you can use $k=1$ so e.g. you would compare two mondays and assume their difference to be the underlying trend. Ideally $s=k$ so that for each seasonal variant, there is a "trend".

There are three parameters, α,β and γ

In [ ]:
ops=['add','mul'] # List the options (additive and multiplicative)
hw_combos = itertools.product([True,False],ops,ops) #Create all 8 combos of additive, multiplicative and True/False
list(hw_combos)

In [ ]:
hw_final_df,hw_selected_model_fit=Fit_Selected_HW(full_df=FTS_df,\
training_start='2019-11-10',\
forecast_start='2020-02-29 19:00:00',\
forecast_end='2020-03-05 12:00:00',\
var_to_forecast='N',periods_seasonality=168)

hw_selected_model_fit.summary()

In [ ]:
hw_forecast = hw_selected_model_fit.predict(start='2020-02-29 19:00:00', end='2020-03-05 12:00:00')
hw_actuals= FTS_df['2020-02-29 19:00:00':'2020-03-05 12:00:00']


actual_vs_forecast=pd.DataFrame(hw_final_df['modelled'].append(hw_forecast)).join(pd.DataFrame(FTS_df['N']))
actual_vs_forecast['2020-02-21':].plot()

In [ ]:
actual_vs_forecast['2020-02-28':]

In [ ]:
hw_uplift=pd.DataFrame(hw_forecast).join(hw_actuals).rename(columns={0:'expected'})
hw_uplift['uplift']=hw_uplift['N']-hw_uplift['expected']
hw_uplift

In [ ]:
hw_uplift.uplift.sum()

In [ ]:
hw_uplift['2020-03-05':'2020-03-06'].uplift.sum()

In [ ]:
actual_vs_forecast[['actual','expected']]['2020-02-28':'2020-03-05'].plot()

# ARIMA
Based on the Autocorrelation Function (ACF) and is more refined than a moving averages method.
To build an ARIMA, there are a number of steps to take, in addition to those used in e.g. a Moving Averages method.

## ARIMA Step 1: Autocorrelation
Auto correlation is the correlation between an observation and another observation in the same series at a different point in time, e.g. between $Y_t$ and $Y_{t-7}$.

The Autocorrelation Function (ACF) is the full distribution of correlation coefficients up to $t-N$ moments in time ago.
However the ACF doesn't tell the full story. Many $Y_t$ values will be highly correlated with $Y_{t-1}$, for example. And so that means that because $Y_{t-1}$ is highly correlated with $Y_{t-2}$, there will also be a decent agreement between $Y_t$ and $Y_{t-2}$. This can mask true repeating correlations. 

Therefore we have the:
Partial Autocorrelation Function (PACF).

This produces an ACF that removes the effect of other time lags and produces something independent. This is called "partialling out". The PACF is calculated using an AutoRegression (AR), which is literally a regression on the previous Yt-n values. PACF is derived from the coefficients of that regression model.

In [ ]:
# Create a new clean dataset to test Arima
ARIMA_training=pd.DataFrame(training['N']) #Must specify the "pd.DataFrame" else it'll return a Series which has different properties

# Correlation with itself (yesterday)
print("Correlation at lag 1 is :",ARIMA_training['N'].autocorr())

# Correlation with itself (7 days ago)
print("Correlation at lag 24 is :",ARIMA_training['N'].autocorr(24))

# Correlation with exactly same record (useless sanity check)
print("Correlation at lag 168 is :",ARIMA_training['N'].autocorr(168))

# Plot autoregression to see how it correlates with each lag
autocorrelation_plot(ARIMA_training['N'])
pyplot.show()

# Get Partial ACF
from statsmodels.tsa.stattools import pacf

PACF_values=pd.DataFrame(pacf(ARIMA_training['N'],nlags=200)) #PACF numpy array converted to df
PACF_values.plot() #Plot
pyplot.show()

# In this example it appears a PACF can be >1. We have some erratic values going back beyond 25 instances which mask the pattern
sub_df=PACF_values[0:1000]
sub_df.plot(style='.-',xlim=(0,26))
pyplot.show()

PACF_values

## ARIMA Step 2: White Noise
A forecasting model is deemed 'well suited' if the forecast errors are purely random. If this is the case, the residuals are described as "white noise".
If a time series is white noise, both AC and PAC coefficients are approximately independent and normally distributed with mean 0 and standard deviation of $1/\surd n$.
Hence it is useful to plot ACF and PACF with a range of $+/- 1.96/\surd(n)$. Coefficients outside these boundaries are probably not white noise i.e. significant.


In [ ]:
import math
1.96*math.sqrt(25)

## ARIMA Step 3: Recognising non-stationarity
First you have to examine the time plot. If the mean or variance change with time it isn't stationary.
ACF and PACF also give evidence, as autocorrelations of stationary data drop to zero quite quickly, whilst those for non-stationary data can take a number of lags to do so. The PACF of a non-stationary time series will typically have a large coefficient close to 1 at lag 1.

## ARIMA Step 4: Removing non-stationarity- by Differencing
It is important to remove trends from time series data prior to model building since such autocorrelations dominate the ACF. This can be done by differencing. 
A differenced series is the change in each observation in a time series e.g. $Y_t'= Y_t - Y_{t-1}$



Occasionally, removing such differences still doesn't result in non-stationarity, i.e. there is a trend in the changes. Ordinarily, a second round of differencing does the trick i.e. $Y_t''= Y_t'- Y_{t-1}'$
Basically you difference the already-differenced data.

When the differenced data is effectively white noise you know you've done it enough.


### Seasonal Differencing

Seasonal Differencing is the same process but rather than taking Yt- Yt-1, you do Yt- Yt-n where n= the number of periods after which seasonality plays a part, e.g. 7 days in a week, 12 months in a year. It can be repeated to obtain second order seasonal differencing but this is rarely needed.

## ARIMA Step 5: Unpicking an ARIMA Model

ARIMA stands for AR - I - MA
or 
Autoregression - I (differences) - Moving Average

#### AR
An AR model basically predicts $Y_t$ using a regression of  
$ β_1 Y_{t-1} + β_2 Y_{t-2} + ... β_p Y_{t-p} + c + ε_{t} $

It takes one parameter ($p$)- which is the number of time lags you use to make a the prediction.

There are constraints on the coefficients:
- If $p=1$ then $-1 < β_1 < 1$
- If $p=2$ then $-1 < β_2 < 1$  AND $β_1+β2<1$ AND $β_2-β_1<1$
- If $p=3$ even more conditions hold

#### I
The "I" in ARIMA details the differencing applied. So if first order differencing is applied, then the parameter($d$)=1. If second order differencing is necessary then the parameter,$d$=2.

#### MA
An MA model, confusingly has nothing to do with Moving Averages in the traditional sense. Instead it predicts $Y_t$ but regressing on the historical errors. The equation looks identical to above in AR except that $Y_{t-k}$ is replaced with $ε_{t-k}$

The parameter dictacting how many time lags to use is called q.
The coefficients are again constrained.
An MA(1) and an AR(1) model are mirror images.

In truth, I'm not sure that an MA model in isolation would do much. However I think the value comes in conjunction with the AR-I parts, as it would help to ensure no systematic error or deviance over time.

#### Combining for an ARIMA model $(p,d,q)$
Combining the above creates a model whereby we have three parameters $(p,d,q)$:
- $p$ is how many times to autoregress, i.e. how many lags on observed values $Y_{t-k}$ to use to build the model. Rarely above 2. A slow decline in ACF but fast decline in PACF characterises an AR(1) model
- $d$ is how many time to difference the data to achieve stationarity, again rarely above 2
- $q$ is how many lags to regress on the error term. An ACF with a single peak at lag 1, and negative PACF values dying down to 0 characterise and MA(1) model.

#### ARIMA $(p,d,q) (P,D,Q)s$ models
Standard $(p,d,q)$ models can also incorporate a seasonal component. In effect I think that the standard $(p,d,q)$ model is regressing on recent observations to predict the next, i.e. yesterday and maybe the day before.
Adding $(P,D,Q)$ adds the seasonal component e.g. last Saturday. 

So for example I can predict tomorrow (Weds) by looking at today (Tuesday) and last Wednesday because of seasonality. 
I think $(P,D,Q)$ perform the same function as $(p,d,q)$ but $s$ periods ago. So $p=1$ would autoregress on yesterday, $P=2$ and $s=7$ would autoregress on 7 and 14 periods ago. I think.


## ARIMA: Step 6: Explaining it and writing it

Convention is that ARIMA models are written using the "backshift" operator, $B$ which shifts data back one period, i.e.
$BY_t =Y_{t-1}$


Two applications of $B$ shift the data back two period. i.e. $B(BY_t) = B^2Y_t = Y_{t-2}$

For monthly data, we can use the notation $B^12$ to shift back to the same month last year:
$B^12Y_t = Y_{t-12}$

Therefore in general, $B^s$ represents "shift back $s$ time periods".

A first order *difference* is represented by $(1-B)$

$Y_t' = Y_t - Y_{t-1} $ and so $= Y_t - BY_t = (1-B)Y_t$

Similarly, a second order difference is represented as $(1-B)^2$

A backshift notation is convenient because terms can be multiplied together to see the combined effect. For example,  a first difference followed by a first seasonal difference can be written as:
$$(1-B)(1-B^S)Y_t = (1-B-B^S +B^{S+1})Y_t =Y_t -Y_{t-1} -Y_{t-S} +Y_{t-S+1}$$

#### An AR model can be written as 

$$(1-\varphi_1B - ... - \varphi_pB^p)Y_t = c + \epsilon_t$$

(where $\varphi$ is the regression coefficient on the previous time lags $B$)


#### An MA model can be written as 

$$ Y_t = c + (1-\theta_1B - ... - \theta_qB^q)\epsilon_t$$

(where $\theta$ is the regression coefficient on the previous error terms $\epsilon$)

#### An ARIMA $(p,d,q)$ model can be written as:

$$(1-\varphi_1B - ... - \varphi_pB^p)(1-B)^dY_t=(1-\theta_1B - ... - \theta_qB^q)\epsilon_t$$

#### An ARIMA $(p,d,q)(P,D,Q)s$ model can be written as:

$$(1-\varphi_1B - ... - \varphi_pB^p)(1-\Phi_1B - ... - \Phi_PB^SP)(1-B)^d(1-B^S)^DY_t$$
$$=(1-\theta_1B - ... - \theta_qB^q)(1-\Theta_1B^S - ... - \Theta_QB^sQ)\epsilon_t$$

As an example, a $(0,1,1)(0,1,1)12$ model can be written as:
$$(1-B)(1-B^{12})Y_t=(1-\theta_1B)(1-\Theta_1B^{12})\epsilon_t$$

# ARIMA Step 7: Measuring it
In addition to the backshift notation, $\sigma$ denotes the standard deviation of the errors, $\epsilon_t$

All the parameters $(\varphi, \theta, \Theta, \Phi) $ etc. have to be estimated. This is done using *ordinary least squares (OLS)* or *maximum likelihood estimation (MLE)*

#### Portmanteau tests for white noise
Insstead of examining each individual autocorrelation coefficient, 'portmanteau' tests can be used which consider the coefficients taken together. An example is the Ljung-Box text which employs the $Q$ statistic:
$$ Q^*=n(n+2)\sum_{k=1}^h(n-k)^{-1}r_k^2$$

where $n$ is the number of observations,
$h$ is the maximum time lag considered, and $r_k$ is the autocorrelation at lag $k$. 

If the residuals of the model are white noise (which you want) $Q^*$ has chi-square $\chi^2$ distribution with $(h-m)$ degrees of freedom, where $m$ is the number of parameters in the model.

If the $Q^*$ statistic lies in the right hand 5% tail of the $\chi^2$ distribution (i.e. P<0.05) it is normally concluded that the data are not white noise.
So you want it to be >0.05 but equally as this is a low bar, it isn't sufficient as a test of model quality.

#### AIC for model comparison
Several models may be a good fit, so we need to distinguish between them. Indeed the more parameters you use, the better your fit will inevitably be, but this could just be overfitting of randomness. AIC is useful because it penalises overfitting in measuring model fit.

$$ AIC = -2LogL + 2m $$
where $L$ is the likelihood and $m= p +q +P +Q$

The **lower** the AIC score the better.


### Let's get modelling ARIMA!

In [ ]:
ARIMA_modelling_results=Fit_ARIMAs(full_df=FTS_df,\
training_start='2019-11-10',\
forecast_start='2020-02-29 19:00:00',\
forecast_end='2020-03-03 00:00:00',\
var_to_forecast='N',seasonality_lag=168)



In [ ]:
ARIMA_modelling_results_df=pd.DataFrame(ARIMA_modelling_results)

In [ ]:
best_AIC=[i for i in ARIMA_modelling_results if i['AIC']==min([i['AIC'] for i in ARIMA_modelling_results])]
best_AIC
best_MAPE=[i for i in ARIMA_modelling_results if i['avg Absolute Percentage error']==min([i['avg Absolute Percentage error'] for i in ARIMA_modelling_results])]
best_MAPE
ARIMA_modelling_results_df.plot.scatter(x='avg Absolute Percentage error',y='AIC')
pyplot.show()

In [ ]:
best_MAPE=[i for i in ARIMA_modelling_results if i['avg Absolute Percentage error']==min([i['avg Absolute Percentage error'] for i in ARIMA_modelling_results])]
best_MAPE

In [ ]:
ARIMA_modelling_results_df.plot.scatter(x='avg Absolute Percentage error',y='AIC')
pyplot.show()

	model	AIC	avg error	avg Squared error	avg Absolute error	avg Percentage error	avg Absolute % error
56	((1, 0, 0), (1, 0, 1, 24))	26070.939874	2.620083	970.455234	16.953621	-0.033887	0.195712

In [ ]:



ARIMA_modelling_results_df[(ARIMA_modelling_results_df['AIC']<26200)\
                           &(pd.DataFrame(ARIMA_modelling_results)['avg Absolute Percentage error']<.2)]\
.sort_values(by='avg Absolute Percentage error')

In [ ]:
final_df,selected_model_fit=Fit_Selected_ARIMAs(full_df=FTS_df,\
training_start='2019-11-10',\
forecast_start='2020-02-28 00:00:00',\
forecast_end='2020-03-03 23:00:00',\
var_to_forecast='N',pdq=(1,0,0),PDQS=(1, 0, 1, 24))

In [ ]:
final_df

In [ ]:
forecast = selected_model_fit.predict(start='2020-02-28 00:00:00', end='2020-03-03 23:00:00')
actuals= FTS_df['2020-02-28 00:00:00':'2020-03-03 23:00:00']



In [ ]:

#pd.DataFrame(final_df['modelled']).append(pd.DataFrame(forecast))
actual_vs_forecast=pd.DataFrame(final_df['modelled'].append(forecast)).join(pd.DataFrame(FTS_df['N']))
actual_vs_forecast['2020-02-27 00:00:00':].plot()

In [ ]:
uplift=pd.DataFrame(forecast).join(actuals).rename(columns={0:'expected'})
uplift['uplift']=uplift['N']-uplift['expected']
uplift.uplift.sum()


In [ ]:
uplift['2020-03-05':'2020-03-06'].uplift.sum()

In [ ]:

gspy.Write_whole_df_to_gsheet(actual_vs_forecast, '', 'vs forecast',\
                              valueInputOption='RAW', append_overwrite='overwrite', picklepath=credentials_path, headers='Y', topleftcell='A1')
 